Before I could go on to make my tool for a week by week and day by day breakdown of the scriptures, I remembered the approaching October 2024 Seminannual Conference of the Church of the Jesus Christ of Latter-day Saints (dated for October 5th and 6th, 2024). I realized that if I could develop a tool for breaking down those talks into daily chunks using dates and such, then I could build on that work when making my Come, Follow Me breakdown tool. 

In order to start that whole endeavor, I needed to, once again, do some webscraping - this time to get the lengths of the talks from the most recent (April 2024) General Conference. 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import glob

In [4]:
os.getcwd()

'd:\\Faith and Religion Stuff\\Come, Follow Me Breakdowns\\come-follow-me-breakdown-builder'

I started by navigating to the Church's website to get the link to the landing page for the April 2024 General Conference. 

In [5]:
apr_2024_gc_link = 'https://www.churchofjesuschrist.org/study/general-conference/2024/04?lang=eng'

From there, I took a look at the html code that holds the page up to see what objects I needed to get the links for. 

In [6]:
response = requests.get(apr_2024_gc_link)
response

<Response [200]>

A 200 response means that my information requester is working and getting data. 

In [7]:
soup = BeautifulSoup(response.content, "html.parser")
content = soup.prettify()  # This formats the HTML in a (more) readable way
print(content)

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title data-react-helmet="true">
   April 2024 general conference
  </title>
  <meta content="4.19.1" name="version"/>
  <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>
  <meta content="" data-react-helmet="true" name="Search.doc-aid">
   <meta content="April 2024 general conference" data-react-helmet="true" name="title">
    <meta content="April 2024 general conference" data-react-helmet="true" name="description">
     <meta content="April 2024 general conference" data-react-helmet="true" property="og:title"/>
     <meta content="website" data-react-helmet="true" property="og:type"/>
     <meta content="https://www.churchofjesuschrist.org/study/eng/general-conference/2024/04/_manifest" data-react-helmet="true" property="og:url"/>
     <link as="script" href="https://www.churchofjesuschrist.org/services/platform/v4/index.js" rel="preload"/>
     <link href="https://foundry.church

Unfortunately, it seems like for this case, all sections of the page (Saturday Morning Session, Sustaining, Church Audit Report, and the talks) are all formatted using the same class and object names - **class="sc-omeqik-0 ewktus list-tile listTile-WHLxI"**. This will be my CSS_Selector, as this is the "heading" to each of the different html objects. I'll be using that class object to get the authors of each talk, the titles, the descriptions, and the links. Using that information, I'll be able to:
* Clean the created dataframe by deleting things like the sustaining, auditing report, and the links to the entire session videos
* Get the number of paragraphs and lines for each paragraph for each talk
* Export that data to my device for using to test and develop my daily reading breakdown tool. 

I need to start by getting that information. I'll start by just copying and pasting the code that I used previously to get links to chapters of the scriptures. I then adapted the code to look for the authors, titles, descriptions, and links to each of the videos linked on that Conference's landing page. Notes in the code itself explain what is happening with every vital line of code.  

In [8]:
# Define the path to the chromedriver executable
chrome_driver_dir = r'D:\\Faith and Religion Stuff\\Come, Follow Me Breakdowns\\chromedriver-win64'
chrome_driver_path = os.path.join(chrome_driver_dir, 'chromedriver.exe')

 # Set up the headless browser options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Set up the Chrome service
service = Service(chrome_driver_path)    
    
# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Establish a try loop that tries to navigate to the provided link and find and store bits of info that we need 
try:
    # Navigate to the page with your elements - in this case the April 2024 General Conference
    driver.get(apr_2024_gc_link)

    # Find all elements with the specified class name
    # Gotta use dots, not spaces, here because CSS considers each of those spaces to be defining a dif class object
    elements = driver.find_elements(By.CSS_SELECTOR, 'a.sc-omeqik-0.ewktus.list-tile.listTile-WHLxI')  

    # Initialize a list to store authors, titles, descriptions, and links
    primary_meta_list = []
    title_list = []
    description_list = []
    href_list = []
    

    # Iterate over each element
    # This for loop will, for all the videos/links to talks on the 2024 General Conference page, run through each of the following operations before moving onto the next
    for element in elements:
        # Try to get the author
        try:
            # Finds and stores the primary meta element (which is the author of the talk or report)
            primary_meta_element = element.find_element(By.CSS_SELECTOR,'p.primaryMeta')
            # Saves the stored author information as text
            primary_meta = primary_meta_element.text
        # If there is no author, save the author as None or Null
        except:
            primary_meta = None
        # Adds the author (or the None) to the list of authors in the appropriate row
        primary_meta_list.append(primary_meta)

        # Try to get the title - every link/video should have a title
        try:
            # Finds and stores the title element (the title of the video, talk, or report)
            title_element = element.find_element(By.CSS_SELECTOR,'p.title')
            # Saves the stored title as text
            title = title_element.text
        # If there is no title, save the title as None - THIS SHOULD NEVER BE THE CASE
        except:
            title = None
        # add the title (or the None) to the list of titles in the appropriate row
        title_list.append(title)

        # Try to get the description - the summary blurb about the video, talk, or report
        try:
            # Finds and stores the description element (the title of the video, talk, or report)
            description_element = element.find_element(By.CSS_SELECTOR,'p.description')
            # Saves the stored description as text
            description = description_element.text
        # If there is no description, save it as None - THIS SHOULD NEVER BE THE CASE
        except:
            description = None
        # add the title (or the None) to the list of descriptions in the appropriate row
        description_list.append(description)

        # Finds and stores the link (or href) to the video, talk, or report
        # This is ultimately going to be the information we use later to get the lengths (in paragraphs) of the talks and the lengths (in lines) of each of those paragraphs
        href = element.get_attribute('href')
        # Adds the stored href to the list of hrefs in the appropriate row
        href_list.append(href)

    # Creates a dataframe to store all the found and stored lists together
    apr_2024_df = pd.DataFrame({
        'Author': primary_meta_list,
        'Title': title_list,
        'Description': description_list,
        'Link': href_list
    })

# If anything doesn't work for some reason, tell why
except Exception as e:
    print(f"An error occurred: {e}")

# After running everything, close the driver we opened to collect the data
finally:
    # Close the browser
    driver.quit()

The cell below, when run after the cell above, demonstrates that the data was collected and saved ***exactly*** the way I wanted it to be. It also shows that there are 39 videos linked to on that landing page. Of these videos:
* 5 are links to watch an entire session.
* 1 is a link to watch the Sustaining of General Authorities 
* 1 is a link to watch the Auditing Report
* The remaining 32 are actual General Conference talks. Everything besides these will be deleted. 

In [9]:
apr_2024_df

,Author,Title,Description,Link
0,None,Saturday Morning Session,The Saturday Morning Session of the 194th Annu...,https://www.churchofjesuschrist.org/study/gene...
1,Dallin H. Oaks,"Sustaining of General Authorities, Area Sevent...","President Oaks presents General Authorities, A...",https://www.churchofjesuschrist.org/study/gene...
2,Jared B. Larson,"Church Auditing Department Report, 2023",Jared B. Larson reads the Church auditing report.,https://www.churchofjesuschrist.org/study/gene...
3,Jeffrey R. Holland,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...
4,J. Anette Dennis,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...
5,Alexander Dushku,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...
6,Ulisses Soares,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...
7,Jack N. Gerard,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...
8,Henry B. Eyring,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...
9,None,Saturday Afternoon Session,The Saturday Afternoon Session of the 194th An...,https://www.churchofjesuschrist.org/study/gene...


Before proceeding with getting the lengths of the talks and their lines, I need to delete the objects from this dataframe that don't need to be there. These are:
* line 0 - Saturday Morning Session
* line 1 - Sustaining
* line 2 - Audit Report
* line 9 - Saturday Afternoon Session
* line 17 - Saturday Evening Session
* line 23 - Sunday Morning Session
* line 31 - Sunday Afternoon Session

In [10]:
# Removes lines 0 - 2
apr_2024_df = apr_2024_df[3:]

# establishes other lines to remove
lines = [9,17,23,31]
# removes those lines
apr_2024_df = apr_2024_df.drop(lines)

# Displays the updated dataframe
apr_2024_df

,Author,Title,Description,Link
3,Jeffrey R. Holland,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...
4,J. Anette Dennis,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...
5,Alexander Dushku,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...
6,Ulisses Soares,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...
7,Jack N. Gerard,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...
8,Henry B. Eyring,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...
10,David A. Bednar,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...
11,Massimo De Feo,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...
12,Brent H. Nielson,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...
13,Jose L. Alonso,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...


I ran the cell below to ensure that the code that collected all this information would work on other conferences. 

In [29]:
# saves the link to the October 2023 General Conference landing page
oct_2023_gc_link = 'https://www.churchofjesuschrist.org/study/general-conference/2023/10?lang=eng'

# Define the path to the chromedriver executable
chrome_driver_dir = r'D:\\Faith and Religion Stuff\\Come, Follow Me Breakdowns\\chromedriver-win64'
chrome_driver_path = os.path.join(chrome_driver_dir, 'chromedriver.exe')

 # Set up the headless browser options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Set up the Chrome service
service = Service(chrome_driver_path)    
    
# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Establish a try loop that tries to navigate to the provided link and find and store bits of info that we need 
try:
    # Navigate to the page with your elements - in this case the October 2023 General Conference
    driver.get(oct_2023_gc_link)

    # Find all elements with the specified class name
    # Gotta use dots, not spaces, here because CSS considers each of those spaces to be defining a dif class object
    elements = driver.find_elements(By.CSS_SELECTOR, 'a.sc-omeqik-0.ewktus.list-tile.listTile-WHLxI')  

    # Initialize a list to store authors, titles, descriptions, and links
    primary_meta_list = []
    title_list = []
    description_list = []
    href_list = []
    

    # Iterate over each element
    # This for loop will, for all the videos/links to talks on the 2024 General Conference page, run through each of the following operations before moving onto the next
    for element in elements:
        # Try to get the author
        try:
            # Finds and stores the primary meta element (which is the author of the talk or report)
            primary_meta_element = element.find_element(By.CSS_SELECTOR,'p.primaryMeta')
            # Saves the stored author information as text
            primary_meta = primary_meta_element.text
        # If there is no author, save the author as None or Null
        except:
            primary_meta = None
        # Adds the author (or the None) to the list of authors in the appropriate row
        primary_meta_list.append(primary_meta)

        # Try to get the title - every link/video should have a title
        try:
            # Finds and stores the title element (the title of the video, talk, or report)
            title_element = element.find_element(By.CSS_SELECTOR,'p.title')
            # Saves the stored title as text
            title = title_element.text
        # If there is no title, save the title as None - THIS SHOULD NEVER BE THE CASE
        except:
            title = None
        # add the title (or the None) to the list of titles in the appropriate row
        title_list.append(title)

        # Try to get the description - the summary blurb about the video, talk, or report
        try:
            # Finds and stores the description element (the title of the video, talk, or report)
            description_element = element.find_element(By.CSS_SELECTOR,'p.description')
            # Saves the stored description as text
            description = description_element.text
        # If there is no description, save it as None - THIS SHOULD NEVER BE THE CASE
        except:
            description = None
        # add the title (or the None) to the list of descriptions in the appropriate row
        description_list.append(description)

        # Finds and stores the link (or href) to the video, talk, or report
        # This is ultimately going to be the information we use later to get the lengths (in paragraphs) of the talks and the lengths (in lines) of each of those paragraphs
        href = element.get_attribute('href')
        # Adds the stored href to the list of hrefs in the appropriate row
        href_list.append(href)

    # Creates a dataframe to store all the found and stored lists together
    oct_2023_df = pd.DataFrame({
        'Author': primary_meta_list,
        'Title': title_list,
        'Description': description_list,
        'Link': href_list
    })

# If anything doesn't work for some reason, tell why
except Exception as e:
    print(f"An error occurred: {e}")

# After running everything, close the driver we opened to collect the data
finally:
    # Close the browser
    driver.quit()

oct_2023_df

,Author,Title,Description,Link
0,None,Saturday Morning Session,None,https://www.churchofjesuschrist.org/study/gene...
1,David A. Bednar,In the Path of Their Duty,Elder Bednar expresses his gratitude for Churc...,https://www.churchofjesuschrist.org/study/gene...
2,Amy A. Wright,Abide the Day in Christ,Sister Wright teaches that with the help of Je...,https://www.churchofjesuschrist.org/study/gene...
3,Robert M. Daines,"Sir, We Would Like to See Jesus",Elder Daines teaches principles to help us bet...,https://www.churchofjesuschrist.org/study/gene...
4,Carlos A. Godoy,For the Sake of Your Posterity,Elder Godoy asks those who have fallen away or...,https://www.churchofjesuschrist.org/study/gene...
5,D. Todd Christofferson,The Sealing Power,Elder Christofferson teaches that the sealing ...,https://www.churchofjesuschrist.org/study/gene...
6,Ian S. Ardern,Love Thy Neighbour,Elder Ardern teaches that humanitarian efforts...,https://www.churchofjesuschrist.org/study/gene...
7,Dallin H. Oaks,Kingdoms of Glory,President Oaks teaches about the kingdoms of g...,https://www.churchofjesuschrist.org/study/gene...
8,None,Saturday Afternoon Session,None,https://www.churchofjesuschrist.org/study/gene...
9,Henry B. Eyring,"Sustaining of General Authorities, Area Sevent...",President Eyring presents the General Authorit...,https://www.churchofjesuschrist.org/study/gene...


The fact that the October 2023 dataframe displays properly shows that I everything is working as it should. This is excellent. 

I think that going forward I'll have to manually remove the videos I don't want every time, but that took almost no time at all, and was incredibly simple, so it shouldn't be a problem. 

Having gotten all the links I needed and having created the dataframe I wanted, I'm ready to move on to a new task: using regular expressions (RegEx) to create primary key identifiers for these talks. That's a whole project for another day, though. 

Over the next few days I need to: 
* use scraping to automatically gather information about the organization to which the speaker belongs (probably requiring RegEx)
* use RegEx to create some primary key identifiers (using names and organizations) for each of these talks (to make everything compatible with SQL)
* get the lengths of the talks (in paragraphs and minutes) and the lengths of each paragraph (in lines)

### 5 Sep 2024

I've decided to start dating my work, just for the sake of not needing to go back and fix and adjust markdowns like the one above that outline what my next plans are. 

To start the day off, after finishing up my learning about Regular Expressions (Regex), I pulled up the html information for the first talk of the April 2024 General Conference to see what information I'd be looking for with each talk. 

In [2]:
jrh_link = 'https://www.churchofjesuschrist.org/study/general-conference/2024/04/13holland?lang=eng'
response = requests.get(jrh_link)
soup = BeautifulSoup(response.text, 'html.parser')
content = soup.prettify()  # This formats the HTML in a (more) readable way
print(content)

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title data-react-helmet="true">
   Motions of a Hidden Fire
  </title>
  <meta content="4.19.1" name="version"/>
  <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>
  <meta content="159161347" data-react-helmet="true" name="Search.doc-aid">
   <meta content="Motions of a Hidden Fire" data-react-helmet="true" name="title">
    <meta content="President Holland teaches about the power of prayer and testifies that God answers every prayer." data-react-helmet="true" name="description">
     <meta content="https://www.churchofjesuschrist.org/imgs/aa8c48c3769ae5cace7d8eb5dfe5fc385846a55d/full/!192%2C/0/default" data-react-helmet="true" property="og:image"/>
     <meta content="Motions of a Hidden Fire" data-react-helmet="true" property="og:title"/>
     <meta content="website" data-react-helmet="true" property="og:type"/>
     <meta content="https://www.churchofjesuschrist.org/study/eng/

It looks like I'll be able to get the longform office or organization to which each speaker belongs by scraping out the `p class="author-role"`, which for Elder Holland is "Acting President of the Quorum of the Twelve Apostles" and for J. Annette Dennis is "First Counselor in the Relief Society General Presidency." That information should be easy to grab, but I'll have to clean it afterwards using Regex unless I can use Regex to grab only what I need. 

Additionally, each paragraph has its `p data-aid="####"` object. Each paragraph also has an `id="p#'` id object, but the body of the talks each starts with `id="p4"`. So I have a few options:
* I can try to grab all the `id=p#` objects after the `<div class="body-block">`, which separates the body of the talk from the title, author, and `p class="kicker"` summary of the talk. 
* I can try to grab all the `p data-aid="####"` objects after and including `id="p4"`.
* I can grab all the `p data-aid="####"` objects and then remove the ones I don't want. 

The first two seem like they'll be the cleaner, though more complicated options. 

However, I'll start by see if/how I can use Regex to grab the `p class="author-role"` information that I want, then go from there. 

### 6 Sep 2024

I wanted to start today off by locating the bit of html code that tells me the length in time of each talk. This is apparently going to be a **much** more difficult process than I anticipated. Dang. After picking through all the html trying to find the audio or the visual, it looks like what I'll have to do is find the `audio preload="metadata" class="sc-a0q09z-1 cQMORe"` and grab the `src="https://..."` link underneath it (for the audio), then navigate to that link to find the length of the audio. 


Well, I thought that would be the case, but after inspecting all of the html code after following such a link, I found that, for some reason, the length of the talk is not displayed anywhere in the code itself. Hm. Time to ask chatgpt for help. 

After spending probably an hour inspecting everything I could on the page, ChatGPT recommended the solution below. If it works, I'll be going through and breaking down what it does piece by piece. 

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Set up the Chrome WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Navigate to the URL
url = "https://www.churchofjesuschrist.org/study/general-conference/2024/04/13holland?lang=eng"
driver.get(url)

# Wait for the page to load completely
driver.implicitly_wait(10)  # Adjust the time if needed

# Locate the video or audio element using JavaScript
video_duration = driver.execute_script("""
    let video = document.querySelector('video');
    if (video) {
        return video.duration;
    } else {
        let audio = document.querySelector('audio');
        if (audio) {
            return audio.duration;
        }
    }
    return null;  // No media element found
""")

# Display the duration in seconds
if video_duration:
    print(f"Duration: {video_duration} seconds")
else:
    print("No video or audio element found on the page")

# Close the browser
driver.quit()

ModuleNotFoundError: No module named 'webdriver_manager'

The above code didn't work because apparently the webdriver-manager no longer automatically downloads with the ChromeDriver, so I need to install it. 

In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


Through a lot of trial and error with ChatGPTs help, I was able to put together the ride series of commands to get the duration of the video, which is excellent. 

I started by importing the necessary packages from Selenium (including the manager I had to install earlier), and then had to set up the driver, which, so far as I can tell "drives" the jupyter notebook to navigate to the web and do things there. In this case, it navigated to Elder Holland's talk, waited a few seconds for the page to completely load, clicked the play button on the video, waited for the video to fully load, and then inspected the javascript to find the duration of the video in seconds. 

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Navigate to the URL
url = "https://www.churchofjesuschrist.org/study/general-conference/2024/04/13holland?lang=eng"
driver.get(url)

# Wait for the page to load completely
driver.implicitly_wait(10)

# Simulate clicking the play button using the class attribute
try:
    # Locate the play button using its class attribute
    play_button = driver.find_element(By.CSS_SELECTOR, "button.sc-1g7hsbc-0.bCKkuP.sc-bvqtyr-3.eKGiZd")
    play_button.click()
    print("Clicked the play button to start the media.")
except:
    print("Play button not found.")

# Wait for the video element to be present in the DOM
try:
    # Wait explicitly for the video to appear in the DOM
    video_element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, 'video'))
    )
    print("Video element found in the DOM.")
except:
    print("No video element found. Trying to find audio...")

    try:
        # Wait for the audio element to be present in the DOM
        audio_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, 'audio'))
        )
        print("Audio element found in the DOM.")
    except:
        print("No audio element found.")

# Wait for a short period to allow the video to start loading
time.sleep(5)

# Print the page source for debugging
print(driver.page_source)  # Print the current page's HTML to see if the video or audio element is present

# Attempt to retrieve the video duration using JavaScript
try:
    video_duration = driver.execute_script("""
        let video = document.querySelector('video');
        if (video) {
            console.log('Video element is present, checking duration...');
            return video.duration;
        } else {
            let audio = document.querySelector('audio');
            if (audio) {
                console.log('Audio element is present, checking duration...');
                return audio.duration;
            }
        }
        return null;  // No media element found
    """)
    
    # Display the duration in seconds
    if video_duration:
        print(f"Duration: {video_duration:.2f} seconds")
    else:
        print("No video or audio element found on the page")
        
except Exception as e:
    print(f"Error retrieving video duration: {e}")

# Close the browser
driver.quit()

Clicked the play button to start the media.
Video element found in the DOM.
<html lang="en" dir="ltr" data-react-helmet="lang" style="--pf-header-height: 111.60000228881836px;"><head class="at-element-marker"><style type="text/css">.truste_caIcon_display {display: block !important;}</style><style type="text/css">.truste_cursor_pointer {cursor: pointer;}.truste_border_none {border: none;}</style><link rel="icon" href="https://www.churchofjesuschrist.org/services/platform/v4/resources/static/image/favicon.ico" sizes="any"><link rel="manifest" href="https://www.churchofjesuschrist.org/services/platform/v4/resources/static/manifest.json">
<script type="application/javascript">window.PlatformService = { langs: [] };</script>
<style>:root { --grey2: #f7f8f8; --grey3: #eff0f0; --grey5: #e0e2e2; --grey10: #d0d3d3; --grey15: #bdc0c0; --grey20: #a9adad; --grey25: #9da1a1; --grey30: #878a8c; --grey35: #676b6e; --grey40: #53575b; --grey60: #3a3d40; --grey90: #0D0F10; --yellow5: #ffd61a; --yellow10

Having worked through a method to get the length of the video of each talk in seconds, I am now in a position to be able to develop a function that will get that as well as the length of each talk in paragraphs and lines and the office to which each speaker belongs. I think I want to get that information before getting the line counts of each paragraph, and attach that information to the dataframe I made above (which has the author, title, description, and link).

The reason I want to do this is to have all the information I need about each talk in one table, information I can then use as I create the different tables displaying the lengths of each of the paragraphs (in lines) of the talks. 

Having identified my next steps, I'm calling it a day on coding today. 